In [1]:
import json, urllib, csv, locale, collections

In [58]:
drinkByName = dict()

with open('../app/static/drinks.json', 'r') as infile:  
    jsonData = json.load(infile)
    drinklst = jsonData['drinks']

for drink in drinklst:
    drinkByName[drink['name']] = drink

In [59]:
print(len(drinkByName))

2129


In [109]:
def in_db(drink):
    return not (drinkByName.get(drink['name'], False) == False)

def add_to_db(drink):
    name = drink['name']
    if (drinkByName.get(name, False)): # if drink in db
        old = drinkByName.get(name, False)
        for key, value in drink.items():
            # update check
            oldvalue = old.get(key, False)
            if (
                oldvalue != value and ( # old value not same as new
                oldvalue == False # old doesnt have key
                or oldvalue == [] # or empty data
                or oldvalue == ''
                or oldvalue == None
                or oldvalue == {}
                )):
                print(old)
                drinkByName[name] = drink
                return True
    else: # not already in db
        drinkByName[name] = drink
        return True
    
    return False

In [110]:
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

def mr_boston():
    with open('mr-boston-flattened-categories.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        line_count = 0
        add_count = 0
        for row in reader:
            line_count += 1
            
            if line_count == 1:
                print(row, len(row))
                print(row[18:], len(row[18:]))
                
            else:

                name = row[0]

                ingredients = list()
                for i in range(0, 6):
                    ing = (row[3+i*2], row[2+i*2])
                    ingredients.append(ing)
                    
                categories = list()
                for i in range(18, 37):
                    if (row[i] != ''):
                        categories.append(row[i].lower())

                drink = dict()
                drink['name'] = name
                drink['ingredients'] = ingredients
                drink['description'] = row[14]
                drink['src'] = "http://worldartsme.com/images/cocktail-glass-clipart-1.jpg"
                
                drink['reviews'] = []
                drink['rating'] = -1
                
                drink['categories'] = categories

                #drink = DrinkInfo(name=name, ingredients=ingredients, instructions=row[14], #glass=(row[15], row[16]),
                #                 src = "http://worldartsme.com/images/cocktail-glass-clipart-1.jpg")
                
                add_to_db(drink)
                add_count += 1
        print(add_count)

In [111]:
mr_boston()
print(len(drinkByName))

['name', 'category', 'measurement-1', 'ingredient-1', 'measurement-2', 'ingredient-2', 'measurement-3', 'ingredient-3', 'measurement-4', 'ingredient-4', 'measurement-5', 'ingredient-5', 'measurement-6', 'ingredient-6', 'instructions', 'glass', 'glass-size', '', 'Fall', 'Winter', 'Spring', 'Summer', 'Morning', 'Afternoon', 'Night', 'Party', 'Dinner Party', 'Chinese New Year', 'St. Patricks Day', 'Thanksgiving', 'Halloween', 'Sports', 'Easter', 'Graduation', 'New Years', 'Birthday', 'Christmas'] 37
['Fall', 'Winter', 'Spring', 'Summer', 'Morning', 'Afternoon', 'Night', 'Party', 'Dinner Party', 'Chinese New Year', 'St. Patricks Day', 'Thanksgiving', 'Halloween', 'Sports', 'Easter', 'Graduation', 'New Years', 'Birthday', 'Christmas'] 19
990
2200


In [112]:
with open('all_drinks.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    line_count = 0
    dup_count = 0
    add_count = 0
    for row in reader:
        line_count += 1
        #if line_count == 1:
        #    print(row)
            
        if line_count != 1:
            
            name = row[1]
            
            ingredients = list()
            ing = (row[9], row[25])
            ingredients.append(ing)
            for i in range(7,15):
                ing = (row[9+i], row[25+i])
                ingredients.append(ing)
            for i in range(1, 7):
                ing = (row[9+i], row[25+i])
                ingredients.append(ing)
                
            src = row[6]

            drink = dict()
            drink['name'] = name
            drink['ingredients'] = ingredients
            drink['description'] = row[24]
            drink['src'] = src
            
            drink['reviews'] = []
            drink['rating'] = -1
            
            add_to_db(drink)
            add_count += 1
        
    print(dup_count, add_count, len(drinkByName))
            
        

0 546 2200


In [127]:
## import glob, os
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time, random

def drink_scraper(url):
    page = urlopen(url)

    soup = BeautifulSoup(page, 'html.parser')
    
    drink = dict()
    
    name = soup.find('h1', attrs={'id':"recipe-main-content"})
    drink['name'] = name.text.strip()
    
    # REMOVE THIS IF YOU WANT TO UPDATE REVIEWS
    # WILL SKIP DRINKS ALREADY IN DB
    if(in_db(drink)): 
        print('SKIPPING ',drink['name'])
        return False
    
    photo = soup.find('img', attrs={'class':'rec-photo'})
    drink['src'] = photo['src']
    
    drink['ingredients'] = list()
    
    for i in soup.find_all('span', attrs={'itemprop':'recipeIngredient'}):
        drink['ingredients'].append(i.text.strip())
        
    instr = list()
    
    for d in soup.find_all('span', attrs={'class':'recipe-directions__list--item'}):
        instr.append(d.text.strip())
        
    drink['description'] = '\n'.join(instr)
    
    staravg = soup.find('div', attrs={'class':'rating-stars'})
    
    drink['rating']= staravg['data-ratingstars']
    
    reviews = list()
    
        #for x in soup.find_all('div'): print(x)
    for r in soup.find_all('div', attrs={'class':'review-container clearfix'}):
        
        rating = dict()
        
        review = r.find('a', attrs={'class':"review-detail__link"})
        
        reviewURL = urlopen(review['href'])
        rsoup = BeautifulSoup(reviewURL, 'html.parser')
        
        stars = rsoup.find('div', attrs={'class':'review-detail__stars'})
        stars = stars.find('span')
        
        rating['stars'] = stars['data-ratingstars']
        
        body = rsoup.find('p', attrs={'itemprop':'reviewBody'})
        rating['body'] = body.text.strip()
        
        reviews.append(rating)
        
        time.sleep(random.randint(0,2))
        
        
    drink['reviews'] = reviews
    
    return drink
    
    # For now, this version takes precedence
    #if drink['name'] in drinkByName:
        #print(drinkByName[drink['name']])
        #drinkByName[drink['name']]['reviews'] = drink['reviews']
        #drinkByName[drink['name']]['rating'] = drink['rating']
    #drinkByName[drink['name']] = drink

In [129]:
def scrape_category_page(page):
    soup = BeautifulSoup(page, 'html.parser')
    
    drinks = list()
        
    for drinkpage in soup.find_all('article', attrs={'class':'fixed-recipe-card'}):
        try:
            a = drinkpage.find('div', attrs={'class':'grid-card-image-container'})
            href = a.find('a', href=True)
            drink = drink_scraper(href['href'])
            if(not (drink == False) and add_to_db(drink)):
                print(drink['name'])
            elif (not (drink == False)):
                print('DUP:', drink['name'])
        except:
            print("FAILED ON DRINK: "+href['href'])
        
        time.sleep(random.randint(1,4))
        
def scrape_category():
    i = 27 #!!!! ### 42
    while True: ###
        p = urlopen("https://www.allrecipes.com/recipes/133/drinks/cocktails/?page=" + str(i))
        scrape_category_page(p)
        i += 1
        print(i)

#https://www.allrecipes.com/recipes/133/drinks/cocktails/
scrape_category()

SKIPPING  Easy Watermelon Martini
SKIPPING  Diplomat Cocktail


KeyboardInterrupt: 

In [130]:
#print(drinkByName['Mai Tai']['reviews'][1])
#p = urlopen("https://www.google.com/")
print(len(drinkByName))

2422


In [131]:
jsondata = {}
jsondata['drinks'] = []
for drink in drinkByName.values():
    try:
        ing = [(b +" " + a) for (a,b) in drink['ingredients'] if a != ""]
    except ValueError:
        ing = [a for a in drink['ingredients'] if a != ""]
        
    if 'instructions' in drink:
        drink['description'] = drink['instructions']
    
    jsondata['drinks'].append({
        'name' : drink['name'],
        'description' : drink['description'],
        'src' : drink['src'],
        'ingredients' : ing,
        'rating' : drink['rating'],
        'reviews' : drink.get('reviews', []),
        'categories' : drink.get('categories', []),
    })

In [132]:
with open('../app/static/drinks.json', 'w') as outfile:  
    json.dump(jsondata, outfile)